In [111]:
import sqlite3

In [112]:
con = sqlite3.connect('test.db')


In [113]:
cursor = con.cursor()

cursor.execute("""
                SELECT name 
                FROM sqlite_master 
                WHERE type='table' 
                    """)


tables = cursor.fetchall()
print(tables)

[('history',)]


In [114]:
with con:
    data = con.execute("""
                        PRAGMA table_info(history);
                       """)
    for row in data:
        print(row[:3])

(0, 'issue_key', 'text')
(1, 'status', 'text')
(2, 'minutes_in_status', 'int4')
(3, 'previous_status', 'text')
(4, 'started_at', 'TIMESTAMP(35)')
(5, 'ended_at', 'TIMESTAMP(35)')


# SQL 1

In [115]:
with con:
    data = con.execute("""
             
            SELECT 
                DISTINCT issue_group,
                ROUND(avg(sum_h_per_issue) OVER (partition by issue_group ), 2) as avg_hours_spent

            FROM
            (
                SELECT DISTINCT issue_key, issue_group, sum_h_per_issue
            
            
                FROM (
                
                SELECT issue_key, status, minutes_in_status,
                SUM((ended_at - started_at)/3600000) OVER (partition by issue_key) as sum_h_per_issue,
                SUBSTR(issue_key, 1, 1) as issue_group
                

                FROM history

                WHERE status = 'Open'

                )tmp1

            )tmp2

                         """)
    for row in data:
        print(row)

# Структура

# получаем сумму часов в статусе 'open' потраченных на каждую отдельную задачу -> 
# -> суммы уже посчитаны, и дальше нам нужны только уникальные значения для рассчетов ->
# -> оставляем уникальные группы, считаем среднее для каждой группы

('A', 247.33)
('B', 280.87)
('C', 594.79)
('D', 841.46)
('E', 817.17)


# SQL 2

In [116]:
with con:
    data = con.execute("""

                        
            SELECT issue_key, status, DATETIME(started_at / 1000, 'unixepoch')

            FROM
            (
                SELECT *,
                ROW_NUMBER() OVER (PARTITION BY issue_key ORDER BY started_at DESC) AS rn
                

                FROM history

                WHERE DATETIME(started_at / 1000, 'unixepoch')  < date('now') --место для указания даты
            
            )tmp1
            
            WHERE rn = 1
            and status <> 'Closed'
            and status <> 'Resolved'

                       """)
    for row in data:
        print(row)

# Структура

# Для каждой отдельной задачи находим количество строк, в которых фигурирует задача
# и, сортируя по времени в обратном порядке, в  качестве 1 получаем последнюю запись ->
# -> оставляем задачу, статус, время в привычном виде

#указал место, где нужно менять дату, чтобы можно было смотреть на статус задач в прошлом
#поддерживается как формат обычной даты:  '2022-11-28' 
#так и формат со временем, полный формат: 'yyyy-mm-dd hh:mm:ss'

('A-31053', 'Open', '2020-10-21 15:08:13')
('A-32133', 'Open', '2021-04-29 09:55:29')
('A-37761', 'In Owl', '2022-12-28 08:18:46')
('A-42030', 'Open', '2022-09-01 15:04:15')
('A-44107', 'In Owl', '2022-12-26 18:31:23')
('A-44901', 'In Owl', '2022-12-27 12:11:12')
('A-44916', 'Blocked', '2022-09-15 09:17:04')
('A-47808', 'Reopened', '2022-10-31 08:54:08')
('A-49140', 'Reopened', '2022-12-29 11:03:37')
('A-49829', 'In Review', '2022-10-20 10:13:04')
('A-50359', 'In Owl', '2022-11-15 06:51:48')
('A-50422', 'In Testing', '2022-11-01 11:56:49')
('A-51466', 'Waiting for integration', '2022-11-14 12:42:43')
('A-52727', 'Waiting for integration', '2022-12-27 12:11:11')
('A-52884', 'Blocked', '2022-11-28 15:12:43')
('A-53409', 'Waiting for integration', '2022-12-12 15:08:39')
('A-53444', 'Open', '2022-12-07 09:19:27')
('A-53602', 'Waiting for integration', '2022-12-26 13:03:29')
('A-53676', 'Waiting for integration', '2022-12-26 18:31:22')
('A-54115', 'Open', '2022-12-20 17:57:23')
('A-54133', 